# Fake News Project
The goal of this project is to create a fake news prediction system. Fake news is a major problem that can have serious negative effects on how people understand the world around them. You will work with a dataset containing real and fake news in order to train a simple and a more advanced classifier to solve this problem. This project covers the full Data Science pipeline, from data processing, to modelling, to visualization and interpretation.
## Part 1 Data Processing

### Task 1

In [5]:
import pandas as pd
import re
import nltk
nltk.download('stopwords')
import nltk.corpus
import nltk.tokenize 
from nltk.stem import PorterStemmer
from functools import reduce
from cleantext import clean

def clean_text(text):
  clean_text = re.sub(r'([A-Z][A-z]+.?) ([0-9]{1,2}?), ([0-9]{4})', '<DATE>', text)
  clean_text = clean(clean_text,
    lower=True,
    no_urls=True, replace_with_url="<URL>",
    no_emails=True, replace_with_email="<EMAIL>",
    no_numbers=True, replace_with_number="<NUM>",
    no_currency_symbols=True, replace_with_currency_symbol="<CUR>",
    no_punct=True, replace_with_punct="",
    no_line_breaks=True 
  )
  return clean_text

def process_text(text):

  stop_words = set(nltk.corpus.stopwords.words('english'))
  word_tokens = nltk.tokenize.word_tokenize(text)

  # return word_tokens
  # filtered = []
  # for w in word_tokens:
  #   if w not in stop_words:
  #     filtered.append(w)

  # ps = PorterStemmer()
  # for w in filtered:
  #   w = ps.stem(w)
  # print(filtered)




[nltk_data] Downloading package stopwords to
[nltk_data]     /home/katikistan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
df = pd.read_csv('news_sample.csv')
clean_df = df.copy()

clean_df.content = clean_df.content.apply(clean_text)
clean_df.content = clean_df.content.apply(process_text)
clean_df.head() 


<class 'str'>


LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/home/katikistan/nltk_data'
    - '/home/katikistan/miniconda3/nltk_data'
    - '/home/katikistan/miniconda3/share/nltk_data'
    - '/home/katikistan/miniconda3/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************
